## EDA - Brasileirão Pontos Corridos (2003 - 2024)


# Objetivo

Essa análise tem como objetivo obter mais detalhes e conhecer mais os dados do Campeonato Brasileiro de Futebol desde que passou a ser disputado no formato de pontos corridos, em 2003, até a última edição concluída, em 2024. Alguns dos objetivos são:

Identificar os times que participaram do campeonato e as quantidades de participações.

Gerar as tabelas de cada edição.

Identificar as melhores campanhas, a quantidade de títulos, rebaixamentos e saldos de gols dos times. 

Ver as participações dos clubes no Brasileirão por estado e por região.

# Bibliotecas necessárias

In [25]:
import pandas as pd


# Times e quantidades de participações

In [23]:
df = pd.read_csv(r"archive/campeonato-brasileiro-full.csv", sep=',')
df.head()

times = {}
for index, row in df.iterrows():
    if row['rodata'] == 1:
        if row['mandante'] in times:
            times[row['mandante']] += 1
        else:
            times[row['mandante']] = 1
        if row['visitante'] in times:
            times[row['visitante']] += 1
        else:
            times[row['visitante']] = 1

times_ordenado_temp = sorted(times.items(), key = lambda x : x[1], reverse = True)
times_ordenado = dict(times_ordenado_temp)

print(f'Ao todo, {len(times)} times participaram do Brasileirão de 2003 a 2024.')
for time in times_ordenado:
    print(f'{time} tem {times_ordenado[time]} participações.')

Ao todo, 45 times participaram do Brasileirão de 2003 a 2024.
Flamengo tem 22 participações.
Fluminense tem 22 participações.
Sao Paulo tem 22 participações.
Athletico-PR tem 21 participações.
Internacional tem 21 participações.
Santos tem 21 participações.
Corinthians tem 21 participações.
Atletico-MG tem 21 participações.
Gremio tem 20 participações.
Palmeiras tem 20 participações.
Cruzeiro tem 19 participações.
Botafogo-RJ tem 19 participações.
Vasco tem 17 participações.
Coritiba tem 15 participações.
Goias tem 15 participações.
Bahia tem 12 participações.
Vitoria tem 11 participações.
Figueirense tem 11 participações.
Sport tem 11 participações.
Ponte Preta tem 9 participações.
Fortaleza tem 9 participações.
Juventude tem 8 participações.
Atletico-GO tem 8 participações.
Avai tem 7 participações.
Ceara tem 7 participações.
Chapecoense tem 7 participações.
Parana tem 6 participações.
America-MG tem 6 participações.
Criciuma tem 5 participações.
Nautico tem 5 participações.
Braganti

# Construindo a tabela para cada ano

In [ ]:
# Criando a coluna ano para facilitar
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
df['edicao'] = df['data'].dt.year.astype(str)
df.loc[(df['edicao'] == '2021') & (df['data'].dt.month <= 2), 'edicao'] = '2020'
df.head()

# Dividindo df em anos
anos = range(2003, 2025)
partidas_por_ano = {ano: df[df['edicao'] == str(ano)] for ano in anos}

tabelas_por_ano = {}
for edicao in anos:
    resultados_ano = {}
    partidas = partidas_por_ano[edicao]
    times_ano = set(partidas['mandante'])
    for time in times_ano:
        resultados_ano[time] = {
            'pontos': 0,
            'jogos': 0,
            'vitorias': 0,
            'empates': 0,
            'derrotas': 0,
            'gols_pro': 0,
            'gols_contra': 0,
            'saldo': 0,
            'aprov': 0,
            'aprov_mandante': 0,
            'aprov_visitante': 0
        }
    for _, row in partidas.iterrows():
        mandante = row['mandante']
        visitante = row['visitante']
        mandante_placar = row['mandante_Placar']
        visitante_placar = row['visitante_Placar']
        vencedor = row['vencedor']
        
        #mandante
        resultados_ano[mandante]['jogos'] += 1
        resultados_ano[mandante]['gols_pro'] += mandante_placar
        resultados_ano[mandante]['gols_contra'] += visitante_placar
        if mandante == vencedor:
            resultados_ano[mandante]['pontos'] += 3
            resultados_ano[mandante]['vitorias'] += 1
            resultados_ano[mandante]['aprov_mandante'] += 3
        elif vencedor == '-':
            resultados_ano[mandante]['pontos'] += 1
            resultados_ano[mandante]['empates'] += 1
            resultados_ano[mandante]['aprov_mandante'] += 1
        else:
            resultados_ano[mandante]['derrotas'] += 1
        
        #visitante
        resultados_ano[visitante]['jogos'] += 1
        resultados_ano[visitante]['gols_pro'] += visitante_placar
        resultados_ano[visitante]['gols_contra'] += mandante_placar
        if visitante == vencedor:
            resultados_ano[visitante]['pontos'] += 3
            resultados_ano[visitante]['vitorias'] += 1
            resultados_ano[visitante]['aprov_visitante'] += 3
        elif vencedor == '-':
            resultados_ano[visitante]['pontos'] += 1
            resultados_ano[visitante]['empates'] += 1
            resultados_ano[visitante]['aprov_visitante'] += 1
        else:
            resultados_ano[visitante]['derrotas'] += 1
       
    for time, stats in resultados_ano.items():
        stats['saldo'] = stats['gols_pro'] - stats['gols_contra']
        stats['aprov'] = stats['pontos'] / (stats['jogos'] * 3) 
        stats['aprov_mandante'] = stats['aprov_mandante'] / (19 * 3)
        stats['aprov_visitante'] = stats['aprov_visitante'] / (19 * 3)
    
    tabelas_por_ano[edicao] = resultados_ano

for edicao in tabelas_por_ano:
    print(f"Tabela do Brasileirão {edicao}")
    df_tabela = pd.DataFrame.from_dict(tabelas_por_ano[edicao], orient='index')
    df_tabela = df_tabela.sort_values(by=['pontos', 'vitorias', 'saldo'], ascending=False)
    df_tabela['posicao'] = range(1, len(df_tabela) + 1) 
    cols = ['posicao'] + [col for col in df_tabela.columns if col != 'posicao']
    df_tabela = df_tabela[cols]
    display(df_tabela)
    print("-" * 40)

Tabela do Brasileirão 2003


,posicao,pontos,jogos,vitorias,empates,derrotas,gols_pro,gols_contra,saldo,aprov,aprov_mandante,aprov_visitante
Cruzeiro,1,100,46,31,7,8,102,47,55,0.724638,0.982456,0.771930
Santos,2,87,46,25,12,9,93,60,33,0.630435,0.894737,0.631579
Sao Paulo,3,78,46,22,12,12,81,67,14,0.565217,0.771930,0.596491
Coritiba,4,73,46,21,10,15,67,58,9,0.528986,0.824561,0.456140
Atletico-MG,5,72,46,19,15,12,76,62,14,0.521739,0.771930,0.491228
Sao Caetano,6,71,46,19,14,13,53,37,16,0.514493,0.877193,0.368421
Internacional,7,70,46,20,10,16,59,57,2,0.507246,0.807018,0.421053
Flamengo,8,66,46,18,12,16,66,73,-7,0.478261,0.824561,0.333333
Goias,9,65,46,18,11,17,78,63,15,0.471014,0.754386,0.385965
Parana,10,65,46,18,11,17,85,75,10,0.471014,0.789474,0.350877


----------------------------------------
Tabela do Brasileirão 2004


,posicao,pontos,jogos,vitorias,empates,derrotas,gols_pro,gols_contra,saldo,aprov,aprov_mandante,aprov_visitante
Santos,1,89,46,27,8,11,103,58,45,0.644928,0.982456,0.578947
Athletico-PR,2,86,46,25,11,10,93,56,37,0.623188,0.929825,0.578947
Sao Paulo,3,82,46,24,10,12,78,43,35,0.594203,0.982456,0.456140
Palmeiras,4,79,46,22,13,11,72,47,25,0.572464,0.719298,0.666667
Sao Caetano,5,77,46,23,8,15,65,49,16,0.557971,0.842105,0.508772
Corinthians,6,74,46,20,14,12,54,54,0,0.536232,0.754386,0.543860
Goias,7,72,46,21,9,16,81,68,13,0.521739,0.929825,0.333333
Juventude,8,70,46,20,10,16,60,66,-6,0.507246,0.649123,0.578947
Internacional,9,67,46,20,7,19,65,58,7,0.485507,0.859649,0.315789
Fluminense,10,67,46,18,13,15,65,68,-3,0.485507,0.754386,0.421053


----------------------------------------
Tabela do Brasileirão 2005


...

----------------------------------------
Tabela do Brasileirão 2023


,posicao,pontos,jogos,vitorias,empates,derrotas,gols_pro,gols_contra,saldo,aprov,aprov_mandante,aprov_visitante
Palmeiras,1,70,38,20,10,8,64,33,31,0.614035,0.771930,0.456140
Gremio,2,68,38,21,5,12,63,56,7,0.596491,0.771930,0.421053
Atletico-MG,3,66,38,19,9,10,52,32,20,0.578947,0.631579,0.526316
Flamengo,4,66,38,19,9,10,56,42,14,0.578947,0.614035,0.543860
Botafogo-RJ,5,64,38,18,10,10,58,37,21,0.561404,0.649123,0.473684
Bragantino,6,62,38,17,11,10,49,35,14,0.543860,0.666667,0.421053
Fluminense,7,56,38,16,8,14,51,47,4,0.491228,0.754386,0.228070
Athletico-PR,8,56,38,14,14,10,51,43,8,0.491228,0.666667,0.315789
Internacional,9,55,38,15,10,13,46,45,1,0.482456,0.561404,0.403509
Fortaleza,10,54,38,15,9,14,45,44,1,0.473684,0.578947,0.368421


----------------------------------------
Tabela do Brasileirão 2024


,posicao,pontos,jogos,vitorias,empates,derrotas,gols_pro,gols_contra,saldo,aprov,aprov_mandante,aprov_visitante
Botafogo-RJ,1,79,38,23,10,5,59,29,30,0.692982,0.719298,0.666667
Palmeiras,2,73,38,22,7,9,60,33,27,0.640351,0.666667,0.614035
Flamengo,3,70,38,20,10,8,61,42,19,0.614035,0.666667,0.561404
Fortaleza,4,68,38,19,11,8,53,39,14,0.596491,0.824561,0.368421
Internacional,5,65,38,18,11,9,53,36,17,0.570175,0.666667,0.473684
Sao Paulo,6,59,38,17,8,13,53,43,10,0.517544,0.684211,0.350877
Corinthians,7,56,38,15,11,12,54,45,9,0.491228,0.666667,0.315789
Bahia,8,53,38,15,8,15,49,49,0,0.464912,0.631579,0.298246
Cruzeiro,9,52,38,14,10,14,43,41,2,0.456140,0.614035,0.298246
Vasco,10,50,38,14,8,16,43,56,-13,0.438596,0.614035,0.263158


----------------------------------------


# Campeões


In [56]:
campeoes = {}

for ano in tabelas_por_ano:
    df_tabela = pd.DataFrame.from_dict(tabelas_por_ano[ano], orient='index')
    df_tabela = df_tabela.sort_values(by=['pontos', 'vitorias', 'saldo'], ascending=False)
    campeao = df_tabela.index[0]
    if campeao in campeoes:
        campeoes[campeao] += 1
    else:
        campeoes[campeao] = 1

print(campeoes)

{'Cruzeiro': 3, 'Santos': 1, 'Corinthians': 4, 'Sao Paulo': 3, 'Flamengo': 3, 'Fluminense': 2, 'Palmeiras': 4, 'Atletico-MG': 1, 'Botafogo-RJ': 1}
